# Exploratory Data Analysis

In [1]:
!pip install protobuf==3.20.*

In [2]:
import wandb
import pandas as pd

# Fetch the artifact

In [3]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)

local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: roboplanet (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# Generate a profile on the entire dataframe

In [4]:
!pip install --upgrade pandas_profiling

In [5]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:38.526221, resuming normal operation.


# Preprocess data
There are missing values in a few columns and the column last_review is a date but it is in string format. The price column has some outliers - there are some zeros and some very high prices. After talking to your stakeholders, you decide to consider from a minimum of  350 per night.

In [6]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

NOTE: we did not impute missing values. We will do that in the inference pipeline, so we will be able to handle missing values also in production.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

# Close the wanb run

In [9]:
run.finish()